<a href="https://colab.research.google.com/github/Seong-jieun/Toy_Project/blob/main/A_Data_Driven_StudyCafe_Experience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

- 분석 배경
    - 취업 준비를 위해 스터디카페를 이용 중인데 비용이 꽤 많이 드는 것 같아서 적절한 이용권과 이용시간을 계산해 볼 필요가 있다는 생각이 들었다

- 분석 목적
    - 이용권을 합리적으로 이용하려면 하루 최소 몇 시간을 이용하면 되는지 계산한다
    - 결제한 이용권이 나의 스터디카페 이용패턴에 맞는 이용권인지 확인한다

- 데이터 정보
    - 데이터 수집 기간: 2024-12-24 ~ 2025-03-10
    - 수집 데이터: 구매 이용권, 이용일자, 입실시간, 퇴실시간, 결제금액, 이용권별 가격
    - 수집 방법:
        - 스터디카페 앱 내 입실, 퇴실 알림, 결제내역을 구글 스프레드시트에 수기 입력했다
        - 외출 여부, 시간은 앱 내에서 확인이 안 되기 때문에 이용시간 내 교통카드 이용내역, 외부에서 결제한 카드내역을 바탕으로 체크했고, 카드 이용 내역이 없는 외출의 경우(ex. 산책) 다이어리 기록, 기억에 의존하여 외출 여부만 체크했다


# 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [ ]:
usedata = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 이용기록.csv')    # 스터디카페 이용기록
payment = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 결제정보.csv')    # 스터디카페 결제정보
price_table = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 가격표.csv')  # 이용권 가격정보

In [ ]:
usedata.head()

,이용권,날짜,입실시간,퇴실시간,외출여부
0,시간권_4시간_1,2024-12-24,13:21:28,17:21:28,x
1,시간권_4시간_2,2024-12-27,13:59:13,17:59:13,x
2,시간권_50시간_1,2024-12-30,13:45:57,19:33:57,x
3,시간권_50시간_1,2025-01-02,11:22:07,19:52:33,x
4,시간권_50시간_1,2025-01-03,11:03:36,18:09:16,x


In [ ]:
payment

,결제이용권,결제일자,시작일시,종료일시,결제금액,이용권금액,비고
0,시간권_4시간_1,2024-12-24,2024-12-24 13:21:00,2024-12-24 17:21:00,"5,000","5,000",NaN
1,시간권_4시간_2,2024-12-27,2024-12-27 13:59:00,2024-12-27 17:59:00,"5,000","5,000",NaN
2,시간권_50시간_1,2024-12-30,2024-12-30 13:45:00,2025-01-13 11:22:00,"70,000","70,000",NaN
3,기간권_4주_1,2025-01-13,2025-01-13 11:21:00,2025-02-10 11:21:00,"130,000","130,000",NaN
4,기간권_4주_2,2025-02-09,2025-02-10 11:21:00,2025-03-10 11:21:00,"120,000","120,000",연장결제
5,기간권_4주_3,2025-03-05,2025-03-10 11:21:00,2025-03-10 11:21:00,"99,000","120,000","연장결제, 할인이벤트, 쿠폰적용"


In [ ]:
price_table

,이용권 구분,이용권,가격,동일권종으로 연장시 가격
0,기간권,2주(14일),"80,000","75,000"
1,기간권,4주(28일),"130,000","120,000"
2,기간권,12주(84일),"350,000","340,000"
3,시간권,50시간(8주),"70,000","65,000"
4,시간권,100시간(16주),"120,000","110,000"
5,시간권,200시간(32주),"220,000","210,000"
6,당일권,2시간,"3,000","3,000"
7,당일권,4시간,"5,000","5,000"
8,당일권,6시간,"6,000","6,000"
9,당일권,8시간,"8,000","8,000"


[컬럼 정보]
- usedata
    - 이용권: 해당 날짜에 적용되는 이용권 종류
    - 날짜: 이용 일자
    - 입실시간: 입실한 시간
    - 퇴실시간: 퇴실한 시간
    - 외출여부: 외출한 경우 o, 외출하지 않은 경우 x (기간권 사용 시에만 외출함)

- payment
    - 결제이용권: 결제한 이용권 종류
    - 결제일자: 이용권을 결제한 일자
    - 시작일시: 이용권 적용 시작 일시
    - 종료일시: 이용권 적용 종료 일시
    - 결제금액: 실제 결제한 금액
    - 이용권금액: 이용권 판매 금액
    - 비고: 참고사항

- price_table
    - 이용권 구분: 이용권은 기간권, 시간권, 당일권으로 구분됨
    - 이용권: 이용권 이름 (시간권은 괄호 안의 기간 안에 시간을 소진해야 함)
    - 가격: 이용권 가격
    - 동일권종으로 연장시 가격: 이용권 연장 구매할 경우 가격 (동일권종은 기간권->기간권, 시간권->시간권일 경우를 의미함)

# 데이터 전처리

In [ ]:
usedata.info(), usedata.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이용권     57 non-null     object
 1   날짜      57 non-null     object
 2   입실시간    56 non-null     object
 3   퇴실시간    57 non-null     object
 4   외출여부    57 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


(None, (57, 5))

In [ ]:
payment.info(), payment.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   결제이용권   6 non-null      object
 1   결제일자    6 non-null      object
 2   시작일시    6 non-null      object
 3   종료일시    6 non-null      object
 4   결제금액    6 non-null      object
 5   이용권금액   6 non-null      object
 6   비고      2 non-null      object
dtypes: object(7)
memory usage: 468.0+ bytes


(None, (6, 7))

In [ ]:
price_table.info(), price_table.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   이용권 구분         11 non-null     object
 1   이용권            11 non-null     object
 2   가격             11 non-null     object
 3   동일권종으로 연장시 가격  11 non-null     object
dtypes: object(4)
memory usage: 484.0+ bytes


(None, (11, 4))

## 데이터 타입 변경

In [ ]:
# usedata 테이블 데이터 타입 변경
usedata['날짜'] = pd.to_datetime(usedata['날짜'], format='%Y-%m-%d')
usedata['입실시간'] = pd.to_datetime(usedata['입실시간'], format='%H:%M:%S', errors='coerce')
usedata['퇴실시간'] = pd.to_datetime(usedata['퇴실시간'], format='%H:%M:%S', errors='coerce')

In [ ]:
# 변경된 타입 확인
usedata.dtypes

,0
이용권,object
날짜,datetime64[ns]
입실시간,datetime64[ns]
퇴실시간,datetime64[ns]
외출여부,object


In [ ]:
# 변경된 테이블 확인
# 입실시간, 퇴실시간에 '1900-01-01'가 같이 출력되지만, 분석에는 시간 데이터만 필요하므로 따로 전처리하지 않기로 한다
usedata.head()

,이용권,날짜,입실시간,퇴실시간,외출여부
0,시간권_4시간_1,2024-12-24,1900-01-01 13:21:28,1900-01-01 17:21:28,x
1,시간권_4시간_2,2024-12-27,1900-01-01 13:59:13,1900-01-01 17:59:13,x
2,시간권_50시간_1,2024-12-30,1900-01-01 13:45:57,1900-01-01 19:33:57,x
3,시간권_50시간_1,2025-01-02,1900-01-01 11:22:07,1900-01-01 19:52:33,x
4,시간권_50시간_1,2025-01-03,1900-01-01 11:03:36,1900-01-01 18:09:16,x


In [ ]:
# payment 테이블 데이터 타입 변경
payment['결제금액'] = payment['결제금액'].str.replace(',', '')
payment['결제금액'] = payment['결제금액'].astype(int)
payment['이용권금액'] = payment['이용권금액'].str.replace(',', '')
payment['이용권금액'] = payment['이용권금액'].astype(int)

In [ ]:
# 변경된 타입 확인
payment.dtypes

,0
결제이용권,object
결제일자,object
시작일시,object
종료일시,object
결제금액,int64
이용권금액,int64
비고,object


In [ ]:
# price_table 테이블 데이터 타입 변경
price_table['가격'] = price_table['가격'].str.replace(',', '')
price_table['가격'] = price_table['가격'].astype(int)
price_table['동일권종으로 연장시 가격'] = price_table['동일권종으로 연장시 가격'].str.replace(',', '')
price_table['동일권종으로 연장시 가격'] = price_table['동일권종으로 연장시 가격'].astype(int)

# 컬럼명 변경
price_table.rename(columns={'동일권종으로 연장시 가격':'연장가'}, inplace=True)

In [ ]:
# 변경된 타입 확인
price_table.dtypes

,0
이용권 구분,object
이용권,object
가격,int64
연장가,int64


## 결측치 처리

In [ ]:
# 결측값이 있는 행 확인
usedata[usedata.isnull().any(axis=1)]

,이용권,날짜,입실시간,퇴실시간,외출여부
31,기간권_4주_1,2025-02-06,NaT,1900-01-01 19:09:47,o


- 결측값 정보
    - 데이터 수집 과정에서 실수로 삭제하면서 입력하지 못한 '입실시간' 값이다

- 결측값 처리 방법
    - `기간권_4주_1`에 해당하면서, `15시 이전에 입실`한 경우의 `평균 입실시간`으로 결측값을 대체하기로 한다

In [ ]:
# '기간권_4주_1'에 해당하는 데이터 필터링
grouped = usedata[usedata['이용권'] == '기간권_4주_1']

# 15시 이전의 입실시간만 필터링 (시간이 15:00:00 이전인 데이터만 선택)
grouped_before_3pm = grouped[grouped['입실시간'].dt.strftime('%H:%M:%S') < '15:00:00']

# 15시 이전 입실시간의 평균 구하기 (시분초까지 포함)
mean_time = grouped_before_3pm['입실시간'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second).mean()

# 평균을 시분초로 다시 변환
mean_hour = mean_time // 3600
mean_minute = (mean_time % 3600) // 60
mean_second = mean_time % 60

# 평균 시간으로 null 값 대체 (시분초 형식으로)
usedata['입실시간'] = usedata.apply(
    lambda row: pd.to_datetime(f"1900-01-01 {int(mean_hour)}:{int(mean_minute)}:{int(mean_second)}")
    if pd.isnull(row['입실시간']) and row['이용권'] == '기간권_4주_1' else row['입실시간'],
    axis=1
)

In [ ]:
# 2025-02-06 일자 행 확인
usedata[usedata['날짜'] == '2025-02-06']

,이용권,날짜,입실시간,퇴실시간,외출여부
31,기간권_4주_1,2025-02-06,1900-01-01 11:41:08,1900-01-01 19:09:47,o


## 파생변수 생성 - 이용시간

In [ ]:
# 이용시간 계산
usedata['이용시간'] = usedata['퇴실시간'] - usedata['입실시간']
usedata.head()

,이용권,날짜,입실시간,퇴실시간,외출여부,이용시간
0,시간권_4시간_1,2024-12-24,1900-01-01 13:21:28,1900-01-01 17:21:28,x,0 days 04:00:00
1,시간권_4시간_2,2024-12-27,1900-01-01 13:59:13,1900-01-01 17:59:13,x,0 days 04:00:00
2,시간권_50시간_1,2024-12-30,1900-01-01 13:45:57,1900-01-01 19:33:57,x,0 days 05:48:00
3,시간권_50시간_1,2025-01-02,1900-01-01 11:22:07,1900-01-01 19:52:33,x,0 days 08:30:26
4,시간권_50시간_1,2025-01-03,1900-01-01 11:03:36,1900-01-01 18:09:16,x,0 days 07:05:40


# EDA

In [ ]:
# 데이터 세트 전체 기간
print(f'이 데이터 세트는 {min(usedata["날짜"].dt.date)}부터 {max(usedata["날짜"].dt.date)}까지의 데이터입니다.')

이 데이터 세트는 2024-12-24부터 2025-03-10까지의 데이터입니다.


In [ ]:
# 구매한 적 있는 이용권
print(f'구매한 적 있는 이용권은 {payment["결제이용권"].unique()}로 총 {payment["결제이용권"].nunique()}종류입니다.')

구매한 적 있는 이용권은 ['시간권_4시간_1' '시간권_4시간_2' '시간권_50시간_1' '기간권_4주_1' '기간권_4주_2' '기간권_4주_3']로 총 6종류입니다.


In [ ]:
# 이용권 구매 횟수와 금액
print(f'이용권 구매 횟수는 {payment["결제이용권"].value_counts().sum()}회 입니다.')
print(f'이용권 구매 금액은 {payment["결제금액"].sum()}입니다.')

이용권 구매 횟수는 6회 입니다.
이용권 구매 금액은 429000입니다.


In [ ]:
# 이용권 구분 확인
usedata['이용권'].unique()

array(['시간권_4시간_1', '시간권_4시간_2', '시간권_50시간_1', '기간권_4주_1', '기간권_4주_2',
       '기간권_4주_3'], dtype=object)

- 이용권별 이용기간, 일 수 확인

In [ ]:
# 시간권_50시간
time_50h_min = min(usedata[usedata["이용권"] == "시간권_50시간_1"]["날짜"].dt.date)
time_50h_max = max(usedata[usedata["이용권"] == "시간권_50시간_1"]["날짜"].dt.date)
time_50h_days = usedata[usedata["이용권"] == "시간권_50시간_1"].value_counts().sum()

# 기간권_4주_1
period_4w_1_min = min(usedata[usedata["이용권"] == "기간권_4주_1"]["날짜"].dt.date)
period_4w_1_max = max(usedata[usedata["이용권"] == "기간권_4주_1"]["날짜"].dt.date)
period_4w_1_days = usedata[usedata["이용권"] == "기간권_4주_1"].value_counts().sum()

# 기간권_4주_2
period_4w_2_min = min(usedata[usedata["이용권"] == "기간권_4주_2"]["날짜"].dt.date)
period_4w_2_max = max(usedata[usedata["이용권"] == "기간권_4주_2"]["날짜"].dt.date)
period_4w_2_days = usedata[usedata["이용권"] == "기간권_4주_2"].value_counts().sum()

print(f'시간권_50시간_1의 이용기간은 {time_50h_min}부터 {time_50h_max}까지이며 {time_50h_days}일 이용했습니다')
print(f'기간권_4주_1의 이용기간은 {period_4w_1_min}부터 {period_4w_1_max}까지이며 {period_4w_1_days}일 이용했습니다')
print(f'기간권_4주_2의 이용기간은 {period_4w_2_min}부터 {period_4w_2_max}까지이며 {period_4w_2_days}일 이용했습니다')

시간권_50시간_1의 이용기간은 2024-12-30부터 2025-01-13까지이며 9일 이용했습니다
기간권_4주_1의 이용기간은 2025-01-13부터 2025-02-09까지이며 23일 이용했습니다
기간권_4주_2의 이용기간은 2025-02-10부터 2025-03-10까지이며 22일 이용했습니다


- 이용권별 총 이용시간, 평균 이용시간, 평균 입퇴실시간 확인

In [ ]:
# '시간권_50시간'
time_50h = usedata[usedata["이용권"] == "시간권_50시간_1"]                       # 데이터프레임 생성
time_50h_time = time_50h["이용시간"].sum()                                       # 총 이용시간
time_50h_avg = str(time_50h["이용시간"].mean()).split(" ")[-1].split(".")[0]     # 평균 이용시간
time_50h_in = str(time_50h["입실시간"].mean()).split(" ")[-1].split(".")[0]      # 평균 입실시간
time_50h_out = str(time_50h["퇴실시간"].mean()).split(" ")[-1].split(".")[0]     # 평균 퇴실시간

# 총 이용시간을 시:분:초 형식으로 변환
t_total_seconds = int(time_50h_time.total_seconds())
hours = t_total_seconds // 3600
minutes = (t_total_seconds % 3600) // 60
seconds = t_total_seconds % 60

# 시:분:초 형식으로 출력
time_50h_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

print("시간권_50시간_1의 총 이용시간:", time_50h_format)
print("시간권_50시간_1의 평균 이용시간:", time_50h_avg)
print("시간권_50시간_1의 평균 입실시간:", time_50h_in)
print("시간권_50시간_1의 평균 퇴실시간:", time_50h_out)

시간권_50시간_1의 총 이용시간: 50:04:26
시간권_50시간_1의 평균 이용시간: 05:33:49
시간권_50시간_1의 평균 입실시간: 12:15:18
시간권_50시간_1의 평균 퇴실시간: 17:49:08


In [ ]:
# '기간권_4주_1'
period_4w_1 = usedata[usedata["이용권"] == "기간권_4주_1"]
period_4w_1_time = period_4w_1["이용시간"].sum()
period_4w_1_avg = str(period_4w_1["이용시간"].mean()).split(" ")[-1].split(".")[0]
period_4w_1_in = str(period_4w_1["입실시간"].mean()).split(" ")[-1].split(".")[0]
period_4w_1_out = str(period_4w_1["퇴실시간"].mean()).split(" ")[-1].split(".")[0]

# 시:분:초 형식으로 변환
p1_total_seconds = int(period_4w_1_time.total_seconds())
hours = p1_total_seconds // 3600
minutes = (p1_total_seconds % 3600) // 60
seconds = p1_total_seconds % 60

# 시:분:초 형식으로 출력
period_4w_1_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

print("기간권_4주_1의 총 이용시간:", period_4w_1_format)
print("기간권_4주_1의 평균 이용시간:", period_4w_1_avg)
print("기간권_4주_1의 평균 입실시간:", period_4w_1_in)
print("기간권_4주_1의 평균 퇴실시간:", period_4w_1_out)

기간권_4주_1의 총 이용시간: 150:24:29
기간권_4주_1의 평균 이용시간: 06:32:22
기간권_4주_1의 평균 입실시간: 12:55:16
기간권_4주_1의 평균 퇴실시간: 19:27:38


In [ ]:
# '기간권_4주_2'
period_4w_2 = usedata[usedata["이용권"] == "기간권_4주_2"]
period_4w_2_time = period_4w_2["이용시간"].sum()
period_4w_2_avg = str(period_4w_2["이용시간"].mean()).split(" ")[-1].split(".")[0]
period_4w_2_in = str(period_4w_2["입실시간"].mean()).split(" ")[-1].split(".")[0]
period_4w_2_out = str(period_4w_2["퇴실시간"].mean()).split(" ")[-1].split(".")[0]

# 시:분:초 형식으로 변환
p2_total_seconds = int(period_4w_2_time.total_seconds())
hours = p2_total_seconds // 3600
minutes = (p2_total_seconds % 3600) // 60
seconds = p2_total_seconds % 60

# 시:분:초 형식으로 출력
period_4w_2_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

print("기간권_4주_2의 총 이용시간:", period_4w_2_format)
print("기간권_4주_2의 평균 이용시간:", period_4w_2_avg)
print("기간권_4주_2의 평균 입실시간:", period_4w_2_in)
print("기간권_4주_2의 평균 퇴실시간:", period_4w_2_out)

기간권_4주_2의 총 이용시간: 137:56:49
기간권_4주_2의 평균 이용시간: 06:16:13
기간권_4주_2의 평균 입실시간: 11:36:17
기간권_4주_2의 평균 퇴실시간: 17:52:30


In [ ]:
# 기간권_4주 출석률
print("기간권_4주_1의 출석률: ", round(23 / 28 * 100, 2),'%')
print("기간권_4주_2의 출석률: ", round(22 / 28 * 100, 2),'%')

기간권_4주_1의 출석률:  82.14 %
기간권_4주_2의 출석률:  78.57 %


In [ ]:
# 전체기간의 평균 입퇴실시간
in_avg = str(usedata['입실시간'].mean()).split(" ")[-1].split(".")[0]
out_avg = str(usedata['퇴실시간'].mean()).split(" ")[-1].split(".")[0]

print("전체기간 평균 입실시간: ", in_avg)
print("전체기간 평균 퇴실시간: ", out_avg)

전체기간 평균 입실시간:  12:18:24
전체기간 평균 퇴실시간:  18:31:04


In [ ]:
# 기간권 외출 횟수
rest_4w_1 = usedata[usedata['이용권'] == '기간권_4주_1'][usedata['외출여부'] == 'o'].value_counts().sum()
rest_4w_2 = usedata[usedata['이용권'] == '기간권_4주_2'][usedata['외출여부'] == 'o'].value_counts().sum()

print(f'기간권_4주_1 외출 횟수: {rest_4w_1}')
print(f'기간권_4주_2 외출 횟수: {rest_4w_2}')

기간권_4주_1 외출 횟수: 8
기간권_4주_2 외출 횟수: 4


In [ ]:
# 외출시간을 제외하면 평균 이용시간이 어떻게 변할까?
# 외출시간 제외하고 대략 계산해보기
print('기간권_4주_1: ', (150 - 8) / 23)
print('기간권_4주_2: ', (145 - 4) / 22)

'''
    - 계산 결과
    - 기간권_4주_1:  6.173913043478261 => 6.17 시간 -> 약 6시간 10분 (22분 감소)
    - 기간권_4주_2:  6.409090909090909 => 6.40 시간 -> 약 6시간 24분 (12분 감소)
'''

기간권_4주_1:  6.173913043478261
기간권_4주_2:  6.409090909090909


'\n    - 계산 결과 \n    - 기간권_4주_1:  6.173913043478261 => 6.17 시간 -> 약 6시간 10분 (22분 감소)\n    - 기간권_4주_2:  6.409090909090909 => 6.40 시간 -> 약 6시간 24분 (12분 감소)\n'

In [ ]:
# 요약된 데이터프레임 만들기
summary = pd.DataFrame({
          '이용권': ['시간권_50시간_1', '기간권_4주_1', '기간권_4주_2']
        , '총 이용시간': [time_50h_format, period_4w_1_format, period_4w_2_format]
        , '평균 이용시간': [time_50h_avg, period_4w_1_avg, period_4w_2_avg]
        , '평균 입실시간': [time_50h_in, period_4w_1_in, period_4w_2_in]
        , '평균 퇴실시간': [time_50h_out, period_4w_1_out, period_4w_2_out]
        , '이용일수': [time_50h_days, period_4w_1_days, period_4w_2_days]
        , '출석률': [100, round(23 / 28 * 100, 2), round(22 / 28 * 100, 2)]
        , '외출횟수': [0, rest_4w_1, rest_4w_2]
        , '결제금액': [payment[payment['결제이용권']=='시간권_50시간_1']['결제금액'].values[0]
                    , payment[payment['결제이용권']=='기간권_4주_1']['결제금액'].values[0]
                    , payment[payment['결제이용권']=='기간권_4주_2']['결제금액'].values[0]]
})

summary

,이용권,총 이용시간,평균 이용시간,평균 입실시간,평균 퇴실시간,이용일수,출석률,외출횟수,결제금액
0,시간권_50시간_1,50:04:26,05:33:49,12:15:18,17:49:08,9,100.00,0,70000
1,기간권_4주_1,150:24:29,06:32:22,12:55:16,19:27:38,23,82.14,8,130000
2,기간권_4주_2,137:56:49,06:16:13,11:36:17,17:52:30,22,78.57,4,120000


[요약]
- 시간권_50시간_1
    - 이용기간은 2024-12-30부터 2025-01-13까지이고 9일동안 50시간을 소진했다
    - 총 이용시간은 50:04:26이다
    - 평균 이용시간은 05:33:49이다
    - 외출횟수는 0회이다
- 기간권_4주_1
    - 이용기간은 2025-01-13부터 2025-02-09까지이고 28일 중 23일 이용했다
    - 총 이용시간은 150:24:29이다
    - 평균 이용시간은 06:32:22이다
    - 출석률은 82.14%이다
    - 외출횟수는 8회이다
- 기간권_4주_2
    - 이용기간은 2025-02-10부터 2025-03-10까지이고 28일 중 22일 이용했다
    - 총 이용시간은 145:33:10이다
    - 평균 이용시간은 06:36:57이다
    - 출석률은 78.57%이다
    - 외출횟수는 4회이다

- 시간권을 사용할 때보다 기간권을 사용할 때 평균 이용시간이 약 1시간 더 늘었다
- 기간권_4주_1과 기간권_4주_2는 이용일수, 총 이용시간, 평균 이용시간 패턴이 유사하다
- 기간권 사용 첫 달(기간권_4주_1)보다 두번째 달(기간권_4주_2)에 총 이용시간과 출석률은 낮아졌다
    - 두번째 달에 하루 덜 갔기 때문으로 보인다
- 외출시간은 정확히 알 수 없으나, 외출 1회당 1시간으로 계산하면 평균 이용시간은 첫 달, 두번째 달에 각각 12분, 22분 줄어든다

- 전체기간 평균 입실시간은 오전 12시 19분이고 평균 퇴실시간은 오후 18시 38분이다

# 분석

- 분석 목적
    - 결제한 이용권이 나의 스터디카페 이용패턴에 합리적인 이용권인지 확인한다
    - 이용권을 합리적으로 이용하려면 하루 최소 몇 시간을 이용하면 되는지 계산한다


## 이용권 대비 이용시간 비교

- 9일동안 50시간을 이용하는 패턴이라면 예상되는 4주 동안 이용시간은 얼마인가?
- 계산식: `28일 X 50시간 / 9일`

In [ ]:
# 28일 X 50시간 / 9일
28 * 50 / 9

155.55555555555554

- 계산결과 9일동안 50시간을 이용하는 패턴으로 4주 기간권을 쓰는 동안 155시간 30분을 이용할 것으로 예상할 수 있다
- 실제 총 이용시간은 `기간권_4주_1`은 150시간 24분, `기간권_4주_2`는 137시간 56분으로 155시간 30분에 못 미친다
- 그렇다면 이용권을 잘못 선택 한 것일까? 금액적인 측면에서 비교해보기로 한다
- 4주(28일)은 9일의 약 3.1배이므로 50시간권을 3번 쓸 경우와 4주기간권을 한 번 쓸 경우 가격을 비교해본다
    - 계산식: `50시간권 최초가격 + 연장시 가격 X 2`

In [ ]:
# 50시간권 최초가격 + 연장시 가격 X 2

70000 + 65000 * 2

200000

50시간권을 3번 썼을 경우 예상되는 비용은 20만원이고 4주 기간권은 13만원이므로 4주 기간권을 선택한 것은 합리적이었던 것으로 보인다

## 이용권 가격대비 최소 이용시간은?

- 앞으로 이용할 것으로 예상되는 이용권 위주로 이용금액 환산
    - 기간권 1일 이용금액
    - 시간권 1시간 이용금액


In [ ]:
price_table

,이용권 구분,이용권,가격,연장가
0,기간권,2주(14일),80000,75000
1,기간권,4주(28일),130000,120000
2,기간권,12주(84일),350000,340000
3,시간권,50시간(8주),70000,65000
4,시간권,100시간(16주),120000,110000
5,시간권,200시간(32주),220000,210000
6,당일권,2시간,3000,3000
7,당일권,4시간,5000,5000
8,당일권,6시간,6000,6000
9,당일권,8시간,8000,8000


In [ ]:
# 기간권 1일 이용금액 환산
price_2w = int(math.ceil(price_table[price_table['이용권']=='2주(14일)']['가격'] / 14))               # 2주 기간권 금액
price_2w_extension = int(math.ceil(price_table[price_table['이용권']=='2주(14일)']['연장가'] / 14))   # 2주 기간권 연장시 금액
price_4w = int(math.ceil(price_table[price_table['이용권']=='4주(28일)']['가격'] / 28))               # 4주 기간권 금액
price_4w_extension = int(math.ceil(price_table[price_table['이용권']=='2주(14일)']['연장가'] / 28))   # 4주 기간권 연장시 금액
price_12w = int(math.ceil(price_table[price_table['이용권']=='12주(84일)']['가격'] / 84))             # 12주 기간권 금액
price_12w_extension = int(math.ceil(price_table[price_table['이용권']=='12주(84일)']['연장가'] / 84)) # 12주 기간권 연장시 금액

print('2주 기간권의 1일 이용금액: ', price_2w)
print('2주 기간권 연장시 1일 이용금액: ', price_2w_extension, '\n')
print('4주 기간권의 1일 이용금액: ', price_4w)
print('4주 기간권 연장시 1일 이용금액: ', price_4w_extension, '\n')
print('12주 기간권의 1일 이용금액: ', price_12w)
print('12주 기간권 연장시 1일 이용금액: ', price_12w_extension, '\n')

# 시간권 1시간 이용금액 환산
price_4h = int(math.ceil(price_table[price_table['이용권']=='4시간']['가격'] / 4))             # 4시간권(당일권) 금액
price_50h = int(math.ceil(price_table[price_table['이용권']=='50시간(8주)']['가격'] / 50))     # 50시간권 금액
price_100h = int(math.ceil(price_table[price_table['이용권']=='100시간(16주)']['가격'] / 100)) # 100시간권 금액

print('4시간권(당일권)의 1시간 이용금액: ', price_4h)
print('50시간권의 1시간 이용금액: ', price_50h)
print('100시간권의 1시간 이용금액: ', price_100h)

2주 기간권의 1일 이용금액:  5715
2주 기간권 연장시 1일 이용금액:  5358 

4주 기간권의 1일 이용금액:  4643
4주 기간권 연장시 1일 이용금액:  2679 

12주 기간권의 1일 이용금액:  4167
12주 기간권 연장시 1일 이용금액:  4048 

4시간권(당일권)의 1시간 이용금액:  1250
50시간권의 1시간 이용금액:  1400
100시간권의 1시간 이용금액:  1200


- 이용권 가격대비 최소 이용시간 계산
    - 기준 이용권: 4주 기간권 기준
    - 이용권 가격: 130,000원 (4주 기간권의 신규 결제시 이용금액)
    - 시간당 요금: 1,400원 (50시간권의 1시간 이용금액)
        - 시간권의 1시간 환산 이용금액 중 제일 큰 금액이므로 사용함
- 총 최소 이용시간
    ```
    총 최소 이용시간 = 이용권 가격 / 시간당 요금
    ```

- 일일 최소 이용시간
    ```
    일일 최소 이용시간 = 이용권 가격 / (이용가능일수 X 시간당 요금)
    ```


In [ ]:
# 총 최소 이용시간
price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / price_50h

92.85714285714286

In [ ]:
# 일일 최소 이용시간
price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / (28 * price_50h)

3.316326530612245

4주 기간권 기준 총 최소 이용시간은 약 92시간 52분, 일일 최소 이용시간은 약 3시간 19분이다

In [ ]:
print(price_table[price_table['이용권']=='2주(14일)']['가격'].values[0] / price_50h)
print(price_table[price_table['이용권']=='2주(14일)']['가격'].values[0] / (14 * price_50h))

57.142857142857146
4.081632653061225


In [ ]:
print(price_table[price_table['이용권']=='12주(84일)']['가격'].values[0] / price_50h)
print(price_table[price_table['이용권']=='12주(84일)']['가격'].values[0] / (84 * price_50h))

250.0
2.9761904761904763


- [번외] 2주 기간권이나 12주 기간권을 사용하는 경우는 아래와 같다
    - 2주 기간권: 총 57시간 9분이상, 하루에 4시간 5분이상
    - 12주 기간권: 총 250시간 이상, 하루에 2시간 59분이상

# 결론

- 4주 기간권을 사용한다면 4주간 총 최소 약 92시간 52분, 하루에 최소 약 3시간 19분 이용하면 본전이다
- 4주 기간권을 구매한 1회차, 2회차의 기록과 비교했을 때 총 이용시간, 하루 평균 이용시간이 최소 이용시간보다 많기 때문에 나의 이용패턴에 맞는 이용권을 선택했다고 볼 수 있다

# 회고

- 실생활과 관련있는 주제를 설정하고 직접 나의 데이터를 수집하고 분석하는 프로젝트는 처음이었는데 흥미로웠다
- 데이터 수집 과정에서 어떤 컬럼을 어떤 형식으로 수집하는 것이 보기에 편한지, 데이터를 다루기에 좋을지 등 고민하는 과정을 통해 데이터 수집 단계에서 설계의 중요성을 알게 되었다
- 총이용시간을 연산하는 부분이 쉽지 않았다
- 외출시간에 대한 데이터가 확보된다면 순수한 이용시간을 계산해 볼 수 있을 것이다
- 총이용시간, 평균이용시간, 일일 사용시간 추이 등 데이터를 활용하여 대시보드를 만들어봐도 좋을 것 같다